# 파일 다루기
- 파일 열기 open(), with로 열기
- csv, xlsx 파일 다루기
- zip 파일 다루기
- 폴더 내 파일 목록 다루기 (glob, listdir)
- 폴더/파일 생성과 삭제

## import

In [1]:
import pandas as pd
import numpy as np
from numpy.random import rand
import zipfile
import csv
import os
from glob import glob

##  데이터  생성과 저장하기
- 숫자 데이터를 생성하고 csv 파일로 저장하기

In [2]:
## 0~1 사이의 랜덤 넘버 10개를 만들고 csv 파일에 저장하고 확인하기
# 텍스트 파일 저장을 위해 savetxt()를 사용한다
# 소수점 몇 째 자리까지 저장할지를 정해주어야 한다 (텍스트로 표현되는)

import numpy as np
x1 = np.random.rand(10)
# print(type(x1))         # ndarray 타입

np.savetxt("test_1.csv", x1, delimiter=",", fmt = '%.3f')

with open('test_1.csv') as f: 
    contents = f.read()
    print(type(contents))  # str 타입
#     print(contents)

<class 'str'>


In [6]:
## csv 파일을 어레이로 읽어서 1000을 곱한 값 보기
# 텍스트 파일을 loadtxt()로 읽으면 어레이로 읽는다 

x = np.loadtxt('test_1.csv', delimiter=',')
print(type(x))    # ndarray 타입
print(x*1000)

<class 'numpy.ndarray'>
[966. 234. 118. 989. 910. 452. 883. 505. 922. 720.]


### 2차원 구조 csv 데이터
- 행이 바뀔 때 리턴 코드가 들어 있다

In [7]:
## 0~30 로 구성된 (5,6) 구조의 어레이를 csv 파일에 저장하기

x2 = np.arange(30).reshape(5,6)

np.savetxt("test_2.csv", x2, delimiter=",", fmt = '%d')

with open('test_2.csv') as f: 
    contents = f.read()
    print(type(contents))  # str 타입
    print(contents)

<class 'str'>
0,1,2,3,4,5
6,7,8,9,10,11
12,13,14,15,16,17
18,19,20,21,22,23
24,25,26,27,28,29



In [8]:
## 2차원 csv 파일을 어레이로 읽는 예

x = np.loadtxt('test_2.csv', delimiter=',')
print(type(x))
print(x)

<class 'numpy.ndarray'>
[[ 0.  1.  2.  3.  4.  5.]
 [ 6.  7.  8.  9. 10. 11.]
 [12. 13. 14. 15. 16. 17.]
 [18. 19. 20. 21. 22. 23.]
 [24. 25. 26. 27. 28. 29.]]


In [9]:
## loadtxt()가 제공하는, 일부 데이터만 읽는 옵션

print('---- 3개의 행만 읽는다 ----')
x = np.loadtxt('test_2.csv', delimiter=',', max_rows=3)
print(x)

print('\n---- 처음 2개의 행은 읽지 않는다 ----')
x = np.loadtxt('test_2.csv', delimiter=',', skiprows=2)
print(x)

print('\n---- 컬럼을 (0,2,4)만 읽는다 ----')
x = np.loadtxt('test_2.csv', delimiter=',', usecols=(0,2,4))
print(x)

---- 3개의 행만 읽는다 ----
[[ 0.  1.  2.  3.  4.  5.]
 [ 6.  7.  8.  9. 10. 11.]
 [12. 13. 14. 15. 16. 17.]]

---- 처음 2개의 행은 읽지 않는다 ----
[[12. 13. 14. 15. 16. 17.]
 [18. 19. 20. 21. 22. 23.]
 [24. 25. 26. 27. 28. 29.]]

---- 컬럼을 (0,2,4)만 읽는다 ----
[[ 0.  2.  4.]
 [ 6.  8. 10.]
 [12. 14. 16.]
 [18. 20. 22.]
 [24. 26. 28.]]


## 데이터프레임으로 읽기

In [9]:
## 2차원 어레이를 데이터프레임으로 읽기

df = pd.read_csv('test_2.csv')  
df

,0,1,2,3,4,5
0,6,7,8,9,10,11
1,12,13,14,15,16,17
2,18,19,20,21,22,23
3,24,25,26,27,28,29


In [10]:
# 어레이로 변환하기
df.values

array([[ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29]])

### read_csv 옵션

- nrows=1000         # 상위 1000줄만 읽기
- skiprows = 3       # 처음 3행 건너뛰기
- skipfooter = 1000 # 맨 뒤의 1000행은 읽지 않기
- usecols= (0,2,4)  # 해당 컬럼만 읽기

In [11]:
# 일부 행만 읽기 nrows 사용 (3줄만 읽기)

df2 = pd.read_csv('test_2.csv', nrows=3, header=None)
df2

,0,1,2,3,4,5
0,0,1,2,3,4,5
1,6,7,8,9,10,11
2,12,13,14,15,16,17


In [12]:
# 3줄 건너뛰기

df2 = pd.read_csv('test_2.csv', skiprows=3, header=None)
df2

,0,1,2,3,4,5
0,18,19,20,21,22,23
1,24,25,26,27,28,29


### 액셀 한글 인코딩
- 문자의 인코딩은 기본적으로 utf-8 을 사용한다
- 그러나 한글의 경우 다른 인코딩으로 저장되는 경우가 있다
 - MS 액셀에서 csv로 저장할 때 한글이 cpc949로 인코딩된다. 이때는 아래 옵션이 필요하다
 - encoding = 'cpc949'
 - encoding = 'euc-kr'을 선택해야 하는 경우도 있다

## zip 파일 
### zip 파일 만들기
- test_1.csv, test_2.csv 두 파일을 output.zip로 압축하는 경우

In [15]:
## zip 파일로 압축하기

with zipfile.ZipFile('output.zip', 'w') as z:
    z.write('test_1.csv')
    z.write('test_2.csv')

# zip에 포함된 파일명 보기와 압출 풀기

with zipfile.ZipFile("output.zip", 'r') as z: 
    print(z.namelist()) 
    z.extractall() 

['test_1.csv', 'test_2.csv']


- (참고) read_csv() 호출시에 zip 파일을 바로 읽을 수 있다 (압축을 풀 필요 없다)

In [16]:
# pd.read_csv('output.zip', header=None)  

# 폴더 작업

## 현재 폴더 위치 보기
- getcwd()를 사용한다
- cwd (current working directory)

In [11]:
## 현재 작업중인 폴더 경로 보기

import os
cwd = os.getcwd()
print(cwd)

C:\Users\User\Dropbox\lab\book13


In [12]:
## 폴더 새로 만들기, mkdir()를 사용한다
# 특정 폴더가 이미 있는지 확인하기 위해서 os.path.exists()를 사용한다
# 아래는 폴더가 없는 경우만 새로 만든다

if not os.path.exists("./data_tmp"):
    os.mkdir("./data_tmp")

## 폴더의 파일 목록 얻기
- glob() 사용: 파일 타입을 편리하게 지정할 수 있다
- listdir() 사용: 경로를 지정할 수 있다

## glob() 사용

In [31]:
## 현재 폴더의 파일 목록 얻기

from glob import glob
f_list = glob('*.ipynb')
f_list

['e_22_Python.ipynb',
 'e_23_Pandas.ipynb',
 'e_24_numpy.ipynb',
 'e_25_Function.ipynb',
 'e_31_FileFolder.ipynb',
 'e_32_Graphs.ipynb',
 'e_41_Preprocessing.ipynb',
 'e_42_Preprocessing_Example.ipynb',
 'e_51_ML_Intro.ipynb',
 'e_52_Regression.ipynb',
 'e_53_Classification.ipynb',
 'e_54_LogisticRegression.ipynb',
 'e_55_DT_RF_Boost.ipynb',
 'e_56_DT_RF_Boost_Reg.ipynb',
 'e_61_FeatureEngineering.ipynb',
 'e_62_Clustering.ipynb',
 'e_63_Add_features.ipynb',
 'e_71_MLP_CNN_MNIST_Keras.ipynb',
 'e_72_CatDog_Keras.ipynb',
 'e_73_TransferLearning.ipynb',
 'e_74_ActivityRecognition.ipynb',
 'e_75_1_YOLOv8_run.ipynb',
 'e_75_2_YOLOv8_Aquarium_Data.ipynb',
 'e_76_1_opencv_face.ipynb',
 'e_76_2_contour_detect.ipynb',
 'e_76_3_opencv_digit.ipynb',
 'e_81_1_TimeseriesForecasting.ipynb',
 'e_81_2_Series_Shift.ipynb',
 'e_82_TimeSeries_XY_bike.ipynb',
 'e_83_LSTM_Intro.ipynb',
 'e_85_prophet.ipynb',
 'e_86_PM_RF_LSTM.ipynb',
 't2.ipynb',
 't3.ipynb',
 'test.ipynb']

In [17]:
# 알파벳 순으로 리스트를 소팅한 결과 보기
# 파일의 순서를 통일시킬 때 필요하다

sorted(f_list)

['e_22_Python.ipynb',
 'e_23_Pandas.ipynb',
 'e_24_numpy.ipynb',
 'e_25_Function.ipynb',
 'e_31_FileFolder.ipynb',
 'e_32_Graphs.ipynb',
 'e_41_Preprocessing.ipynb',
 'e_42_Preprocessing_Example.ipynb',
 'e_51_ML_Intro.ipynb',
 'e_52_Regression.ipynb',
 'e_53_Classification.ipynb',
 'e_54_LogisticRegression.ipynb',
 'e_55_DT_RF_Boost.ipynb',
 'e_56_DT_RF_Boost_Reg.ipynb',
 'e_61_FeatureEngineering.ipynb',
 'e_62_Clustering.ipynb',
 'e_63_Add_features.ipynb',
 'e_71_MLP_CNN_MNIST_Keras.ipynb',
 'e_72_CatDog_Keras.ipynb',
 'e_73_TransferLearning.ipynb',
 'e_74_ActivityRecognition.ipynb',
 'e_75_1_YOLOv8_run.ipynb',
 'e_75_2_YOLOv8_Aquarium_Data.ipynb',
 'e_76_1_opencv_face.ipynb',
 'e_76_2_contour_detect.ipynb',
 'e_76_3_opencv_digit.ipynb',
 'e_81_1_TimeseriesForecasting.ipynb',
 'e_81_2_Series_Shift.ipynb',
 'e_82_TimeSeries_XY_bike.ipynb',
 'e_83_LSTM_Intro.ipynb',
 'e_85_prophet.ipynb',
 'e_86_PM_RF_LSTM.ipynb',
 't2.ipynb',
 't3.ipynb',
 'test.ipynb']

## listdir() 사용
- 인자에 파일을 찾을 폴더 경로를 지정할 수 있다
- 쓰지 않으면 현재 폴더를 찾는다, 모든 파일을 찾아준다

In [23]:
os.listdir()

['.ipynb_checkpoints',
 'big.csv',
 'data_tmp',
 'e_22_Python.ipynb',
 'e_23_Pandas.ipynb',
 'e_24_numpy.ipynb',
 'e_25_Function.ipynb',
 'e_31_FileFolder.ipynb',
 'e_32_Graphs.ipynb',
 'e_41_Preprocessing.ipynb',
 'e_42_Preprocessing_Example.ipynb',
 'e_51_ML_Intro.ipynb',
 'e_52_Regression.ipynb',
 'e_53_Classification.ipynb',
 'e_54_LogisticRegression.ipynb',
 'e_55_DT_RF_Boost.ipynb',
 'e_56_DT_RF_Boost_Reg.ipynb',
 'e_61_FeatureEngineering.ipynb',
 'e_62_Clustering.ipynb',
 'e_63_Add_features.ipynb',
 'e_71_MLP_CNN_MNIST_Keras.ipynb',
 'e_72_CatDog_Keras.ipynb',
 'e_73_TransferLearning.ipynb',
 'e_74_ActivityRecognition.ipynb',
 'e_75_1_YOLOv8_run.ipynb',
 'e_75_2_YOLOv8_Aquarium_Data.ipynb',
 'e_76_1_opencv_face.ipynb',
 'e_76_2_contour_detect.ipynb',
 'e_76_3_opencv_digit.ipynb',
 'e_80_Streamlit.py',
 'e_81_1_TimeseriesForecasting.ipynb',
 'e_81_2_Series_Shift.ipynb',
 'e_82_TimeSeries_XY_bike.ipynb',
 'e_83_LSTM_Intro.ipynb',
 'e_85_prophet.ipynb',
 'e_86_PM_RF_LSTM.ipynb',
 '

In [29]:
os.listdir('c:\Windows')

['addins',
 'AhnInst.log',
 'appcompat',
 'Application Data',
 'apppatch',
 'AppReadiness',
 'assembly',
 'bcastdvr',
 'bfsvc.exe',
 'Boot',
 'bootstat.dat',
 'Branding',
 'CbsTemp',
 'comsetup.log',
 'ConfigSetRoot',
 'Containers',
 'Core.xml',
 'court',
 'csup.txt',
 'Cursors',
 'debug',
 'diagerr.xml',
 'diagnostics',
 'DiagTrack',
 'diagwrn.xml',
 'DigitalLocker',
 'Downloaded Program Files',
 'DtcInstall.log',
 'ELAMBKUP',
 'en-US',
 'explorer.exe',
 'EXPRESSCACHE1.txt',
 'FFS.txt',
 'FFS_0.txt',
 'FFS_1.txt',
 'Firmware',
 'Fonts',
 'GameBarPresenceWriter',
 'Globalization',
 'HancomIME',
 'Help',
 'HelpPane.exe',
 'hh.exe',
 'IdentityCRL',
 'ImageSAFERSvc.exe',
 'IME',
 'IMGSF50Svc.exe',
 'ImmersiveControlPanel',
 'INF',
 'InputMethod',
 'Installer',
 'ko-KR',
 'L2Schemas',
 'LanguageOverlayCache',
 'LGPS',
 'lgpslog',
 'LiveKernelReports',
 'Logs',
 'lsasetup.log',
 'Media',
 'mib.bin',
 'Microsoft.NET',
 'Migration',
 'ModemLogs',
 'notepad.exe',
 'OCR',
 'Offline Web Pages',


## 기타 폴더 및 파일 관리 함수
- os 패키지에서 제공하는 함수
 - 폴더 생성 mkdir()
 - 폴더 위치 이동 chdir()
 - 파일 이름 변경 rename()
 - 파일 삭제 remove()
 - 폴더 삭제 rmdir()